# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [82]:
# your code here
orders = pd.read_csv('../data/orders_sample.csv')
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80
1,545906,22557,2011,3,2,9,plasters in tin vintage paisley,12,2011-03-08 09:23:00,1.65,15764,United Kingdom,19.80
2,539475,22176,2010,12,7,14,blue owl soft toy,1,2010-12-19 14:41:00,2.95,16686,United Kingdom,2.95
3,572562,21889,2011,10,2,9,wooden box of dominoes,12,2011-10-25 09:07:00,1.25,13481,United Kingdom,15.00
4,549372,72741,2011,4,5,11,grand chocolatecandle,9,2011-04-08 11:28:00,1.45,14958,United Kingdom,13.05


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [21]:
#Group by customer ID to check amount_spent per customer:

orders.groupby(by='CustomerID').sum().sort_values(by= 'amount_spent', ascending=False)

,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,
14646,61544094,221206,762,328,1287,10084,230.63,12518.46
18102,15152055,54296,208,76,334,3828,92.82,12167.41
17450,8331900,30164,94,41,182,3268,51.48,11000.00
14156,39097979,140770,449,227,865,3563,284.41,9042.19
12415,30071770,108594,321,159,598,4583,140.82,8844.52
...,...,...,...,...,...,...,...,...
12977,559165,2011,7,3,16,1,0.65,0.65
15578,558555,2011,6,4,13,1,0.55,0.55
15853,571743,2011,10,3,10,1,0.55,0.55


In [37]:
# VIP mask

mask = orders.groupby(by='CustomerID').sum().amount_spent >=orders.groupby(by='CustomerID').sum().amount_spent.quantile(q=0.95)
mask

CustomerID
12347    False
12348    False
12349    False
12350    False
12352    False
         ...  
18276    False
18278    False
18280    False
18283    False
18287    False
Name: amount_spent, Length: 3326, dtype: bool

In [63]:
# VIP dataframe
vip = pd.DataFrame(orders.groupby(by='CustomerID').sum().loc[mask,'amount_spent'])
vip['customer_label']= 'VIP'
vip

,amount_spent,customer_label
CustomerID,,
12362,397.13,VIP
12409,549.30,VIP
12415,8844.52,VIP
12432,517.80,VIP
12433,1019.95,VIP
...,...,...
18139,963.90,VIP
18172,762.10,VIP
18225,439.90,VIP


In [51]:
# Preferred customer mask (0.75 ~ 0.95 quantile)

mask2 = orders.groupby(by='CustomerID').sum().amount_spent >= orders.groupby(by='CustomerID').sum().amount_spent.quantile(0.75,interpolation='lower')
mask2_1 = orders.groupby(by='CustomerID').sum().amount_spent < orders.groupby(by='CustomerID').sum().amount_spent.quantile(0.95,interpolation='higher')
(mask2 & mask2_1).sum()

666

In [64]:
# Preferred customer dataframe
pref = pd.DataFrame(orders.groupby(by='CustomerID').sum().loc[(mask2 & mask2_1),'amount_spent'])
pref['customer_label']= 'PREFERRED'
pref

,amount_spent,customer_label
CustomerID,,
12347,149.90,PREFERRED
12352,126.48,PREFERRED
12356,161.88,PREFERRED
12357,264.30,PREFERRED
12359,324.85,PREFERRED
...,...,...
18245,143.30,PREFERRED
18251,252.00,PREFERRED
18260,118.65,PREFERRED


In [80]:
# DataFrame with Labels

labels_df = pd.concat([vip, pref])
labels_df = labels_df.rename(columns={'amount_spent':'agg_amount_spent'})
labels_df

,agg_amount_spent,customer_label
CustomerID,,
12362,397.13,VIP
12409,549.30,VIP
12415,8844.52,VIP
12432,517.80,VIP
12433,1019.95,VIP
...,...,...
18245,143.30,PREFERRED
18251,252.00,PREFERRED
18260,118.65,PREFERRED


In [84]:
lbld_orders = pd.merge(left=orders, right = labels_df, how= 'left', on = ['CustomerID'])
lbld_orders

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,agg_amount_spent,customer_label
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80,188.40,PREFERRED
1,545906,22557,2011,3,2,9,plasters in tin vintage paisley,12,2011-03-08 09:23:00,1.65,15764,United Kingdom,19.80,144.39,PREFERRED
2,539475,22176,2010,12,7,14,blue owl soft toy,1,2010-12-19 14:41:00,2.95,16686,United Kingdom,2.95,NaN,NaN
3,572562,21889,2011,10,2,9,wooden box of dominoes,12,2011-10-25 09:07:00,1.25,13481,United Kingdom,15.00,132.04,PREFERRED
4,549372,72741,2011,4,5,11,grand chocolatecandle,9,2011-04-08 11:28:00,1.45,14958,United Kingdom,13.05,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,563094,23152,2011,8,4,17,ivory sweetheart wire letter rack,2,2011-08-11 17:35:00,3.75,14525,United Kingdom,7.50,395.78,VIP
19996,578626,21592,2011,11,4,15,retrospot cigar box matches,24,2011-11-24 15:38:00,1.25,12709,Germany,30.00,249.91,PREFERRED
19997,561073,23245,2011,7,7,15,set of 3 regency cake tins,16,2011-07-24 15:46:00,4.15,13408,United Kingdom,66.40,857.36,VIP
19998,576861,20674,2011,11,3,16,green polkadot bowl,12,2011-11-16 16:00:00,1.25,18041,United Kingdom,15.00,164.64,PREFERRED


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [121]:
# United Kingdom has the most VIP customers

# your code here
mask = lbld_orders.loc[:,'customer_label'] == 'VIP'
vip_count = lbld_orders.loc[mask,['Country','customer_label']]
vip_count.groupby(by='Country').count().sort_values(by= 'customer_label',ascending=False)

,customer_label
Country,
United Kingdom,3562
EIRE,375
France,124
Netherlands,110
Germany,94
Australia,54
Spain,50
Norway,41
Switzerland,30


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here

In [108]:
pd.DataFrame(lbld_orders.groupby(by=['Country','customer_label']).count().loc[:,'amount_spent'].sort_values(ascending=False))

amount_spent
Country              customer_label              
United Kingdom       PREFERRED               6535
                     VIP                     3562
EIRE                 VIP                      375
Germany              PREFERRED                261
France               PREFERRED                148
                     VIP                      124
Netherlands          VIP                      110
Germany              VIP                       94
Australia            VIP                       54
Belgium              PREFERRED                 53
Spain                VIP                       50
Norway               VIP                       41
Switzerland          VIP                       30
Cyprus               PREFERRED                 28
Switzerland          PREFERRED                 27
Channel Islands      PREFERRED                 26
Spain                PREFERRED                 26
Finland              PREFERRED                 25
Belgium              VIP                       19
Portugal             PREFERRED                 16
Norway               PREFERRED                 15
Italy                PREFERRED                 15
Japan                VIP                       12
Singapore            VIP                       10
Poland               PREFERRED                  8
Sweden               VIP                        8
Australia            PREFERRED                  7
Iceland              PREFERRED                  6
Canada               PREFERRED                  6
Sweden               PREFERRED                  6
Israel               PREFERRED                  5
EIRE                 PREFERRED                  5
Japan                PREFERRED                  5
Denmark              PREFERRED                  4
Austria              PREFERRED                  4
Israel               VIP                        4
Greece               PREFERRED                  3
Lithuania            PREFERRED                  3
United Arab Emirates PREFERRED                  3
USA                  PREFERRED                  2

In [86]:
lbld_orders.isna().sum()

InvoiceNo              0
StockCode              0
year                   0
month                  0
day                    0
hour                   0
Description            0
Quantity               0
InvoiceDate            0
UnitPrice              0
CustomerID             0
Country                0
amount_spent           0
agg_amount_spent    8265
customer_label      8265
dtype: int64

In [91]:
lbld_orders['agg_amount_spent'].isna()

0        False
1        False
2         True
3        False
4         True
         ...  
19995    False
19996    False
19997    False
19998    False
19999     True
Name: agg_amount_spent, Length: 20000, dtype: bool

In [90]:
orders.groupby(by='CustomerID').sum().loc[:,'amount_spent']

CustomerID
12347    149.90
12348     75.36
12349    100.09
12350     10.20
12352    126.48
          ...  
18276     25.20
18278     29.70
18280     23.60
18283     78.08
18287    209.52
Name: amount_spent, Length: 3326, dtype: float64